In [1]:
#LIBRARIES

# VREP API library
import sim
import sys

# import numpy and 
import numpy as np
import time

In [2]:


def getSensorValues(sc,sl,sr):
    _, _, dPC, _, _=sim.simxReadProximitySensor(clientID,sc, sim.simx_opmode_blocking)
    _, _, dPL, _, _=sim.simxReadProximitySensor(clientID,sl, sim.simx_opmode_blocking)
    _, _, dPR, _, _=sim.simxReadProximitySensor(clientID,sr, sim.simx_opmode_blocking)
    return np.linalg.norm(dPC),np.linalg.norm(dPL),np.linalg.norm(dPR)

def getSensorHandles():
    err,Sensor_Cara=sim.simxGetObjectHandle(clientID,'Sensor_Cara',sim.simx_opmode_blocking)
    err,Sensor_left=sim.simxGetObjectHandle(clientID,'Sensor_left',sim.simx_opmode_blocking)
    err,Sensor_right=sim.simxGetObjectHandle(clientID,'Sensor_right',sim.simx_opmode_blocking)
    return Sensor_Cara, Sensor_left, Sensor_right

# CONNECTION   
def conn():
    #close all connections
    sim.simxFinish(-1)
    # connection with the API
    clientID = -1
    while clientID == -1:
        print("Conectando...")
        clientID = sim.simxStart('127.0.0.1', 19999,True,True,2000,5)
    print("Conectado al puerto 19999")
    return clientID
    
# INITIALIZE ROTORS TO 0
def init_prop(clientID): 
    r,r1,r2,r3,r4 = sim.simxCallScriptFunction(clientID, "Quadricopter", sim.sim_scripttype_childscript,"set_v", [], [0,0,0,0], [] ,"", sim.simx_opmode_blocking)

# MOVING THE DRONE AS AN OBJECT
def move_q(pos):
    err, qcopter = sim.simxGetObjectHandle(clientID, 'Quadricopter_target', sim.simx_opmode_blocking)
    sim.simxSetObjectPosition(clientID, qcopter, -1, pos, sim.simx_opmode_oneshot)

# ROTATE THE DRONE AS AN OBJECT
   

# FUNCTION THAT SETS VELOCITIES TO EACH PROPELER
def move_prop(clientID, vels):
     r,r1,r2,r3,r4 = sim.simxCallScriptFunction(clientID, "Quadricopter", sim.sim_scripttype_childscript,"set_v", [], vels, [] ,"", sim.simx_opmode_blocking)


In [9]:
def in_box(pa,box):
    return (box[0][0] < pa[0] < box[1][0]) and (box[0][1] < pa[1] < box[1][1])

def recta(x,pos_ini,pos_end):
    y=((pos_end[1]-pos_ini[1])/(pos_end[0]-pos_ini[0]))*(x-pos_ini[0])
    return y

def algoritme2(pos_ini,pos_end):
    sc,sl,sr = getSensorHandles()
    pos_actual=pos_ini
    # area de aterrizatge
    box = [[pos_end[0] - 0.3,pos_end[1] - 0.3], [pos_end[0] + 0.3, pos_end[1] +0.3]]
    h_change = 0
    gir=0# 1 dreta,2 esquerra
    # mentre no estem a l'area final(box)
    while not in_box(pos_actual,box):
        # Valors dels sensors
        sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
        # Hi ha alguna cosa davant
        if (sc_v < 0.8 and  sc_v > 0.1 ):
            #Hi ha alguna cosa a l'esquerra i no hi ha res a la dreta, per tant movem a la dreta
            if (sl_v < 0.8 and  sl_v > 0.1):
                # Si també hi ha una cosa a la dreta movem cap amunt
                if (sr_v < 0.8 and  sr_v > 0.1):
                    print("Movent cap amunt")
                    pos_actual[2] = pos_actual[2] + 0.1
                    move_q(pos_actual)
                    h_change=1
                    continue
                # Moviment a la dreta
                print("Movent a la dreta")
                pos_actual[1] = pos_actual[1] -0.1
                move_q(pos_actual)
                gir = 1
            else:
                print("Movent a l'esquerra")
                # Si no hi ha res a l'esquerra Movem cap a l'esquerra
                pos_actual[1] = pos_actual[1] +0.1
                move_q(pos_actual) 
                gir = 2
        
        else:
            print("Sense obstacles, seguim recte")
            # Calculem si el punt actual està a la recta
            y = recta(pos_actual[0],pos_ini,pos_end)
            # Si està a la recta podem seguir endavant, si no és que estem vorejant un objecte
            if pos_actual[1]-0.1 < y < pos_actual[1]+0.1:
                pos_actual[0]=pos_actual[0]+0.1
                y = recta(pos_actual[0],pos_ini,pos_end)
                move_q(pos_actual)
            else:
                # creem una variable per veure si mentre estem avançant trobem un altre objecte davant
                # si es el cas l'hem de vorejar també
                front_obj = 0
                # ara avancem una mica fins detectar l'objecte que estem vorejant
                sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
                # En el cas d'haver canviat d' altura no entrarem
                while (not ((sl_v < 0.8 and  sl_v > 0.1) or (sr_v < 0.8 and  sr_v > 0.1)) and front_obj == 0 and h_change == 0 and in_box(pos_actual,box)==False):
                    print("Endavant fins detectar")
                    # si el sensor frontal detecta una altre objecte 
                    if(sc_v < 0.8 and  sc_v > 0.1):
                        # marquem l'objecte com a detectat
                        front_obj = 1
                        continue
                    pos_actual[0] = pos_actual[0]+0.1
                    move_q(pos_actual)
                    sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
                # si hem detectat un objecte hem de vorejar-lo també, per tant anem a la següent iteració del while general
                if front_obj == 1:
                    continue
                # Un cop haguem detectat l'objecte que estem vorejant fins que no el deixem de detectar seguim endavant també
                while ((sl_v < 0.8 and  sl_v > 0.1) or (sr_v < 0.8 and  sr_v > 0.1) and front_obj == 0 and in_box(pos_actual,box)==False):
                    print("Endavant fins  no detectar")
                    if(sc_v < 0.8 and  sc_v > 0.1):
                        # marquem l'objecte com a detectat
                        front_obj = 1
                        continue
                    pos_actual[0] = pos_actual[0]+0.1
                    move_q(pos_actual)
                    sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
                
                # si hem detectat un objecte hem de vorejar-lo també, per tant anem a la següent iteració del while general
                if front_obj == 1:
                    continue
                # Ara volem tornar a la recta
                #Avancem una mica més per no xocar contra l'objecte
                pos_actual[0] = pos_actual[0]+0.1
                move_q(pos_actual)
                # Ara tenint en compte el gir tornem a la recta
                y = recta(pos_actual[0],pos_ini,pos_end)
                if (gir==1 and in_box(pos_actual,box)==False) :
                    # Tornem desde la dreta
                    print("Tornem desde la dreta")
                    while y > pos_actual[1]:
                        if (sl_v < 0.8 and  sl_v > 0.1):
                            print("Objecte trobat a l'esquerra, tirant endavant")
                            pos_actual[0] = pos_actual[0]+0.1
                            move_q(pos_actual)
                            sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
                            continue
                        print("Tornant...")
                        pos_actual[1] = pos_actual[1]+0.1
                        move_q(pos_actual)
                        sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
                else:
                    if(in_box(pos_actual,box)==False):
                        # Tornem desde la esquerra
                        print("Tornem desde la esquerra",y,pos_actual[1])
                        sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
                        while y < pos_actual[1]:
                            if (sr_v < 0.8 and  sr_v > 0.1):
                                print("Objecte trobat a la dreta, tirant endavant")
                                pos_actual[0] = pos_actual[0]+0.1
                                move_q(pos_actual)
                                sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)
                                continue
                            print("Tornant...")
                            pos_actual[1] = pos_actual[1]-0.1
                            move_q(pos_actual)
                            sc_v,sl_v,sr_v = getSensorValues(sc,sl,sr)

    # Bucle per tornar a la altura inicial
    while pos_actual[2]-pos_end[2] > 0.1:
        pos_actual[2] = pos_actual[2] -0.05
        move_q(pos_actual) 

In [11]:
clientID = conn()
# posició inicial
x = -1.8
y= 0.0
z=0.5
pos_ini = [x,y,z]
move_q(pos_ini)
#posició final
pos_end = [6,y,z]
# crida a l'algorisme
algoritme2(pos_ini,pos_end)

Conectando...
Conectado al puerto 19999
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Sense obstacles, seguim recte
Movent a l'esquerra
Movent a l'esquerra
Movent a l'esquerra
Movent a l'esquerra
Movent a l'esquerra
Movent a l'esquerra
Sense obstacles, seguim recte
Endavant fins detectar
Endavant fins detectar
Endavant fins detectar
Endavant fins detectar
Endavant fins detectar
Endavant fins detectar
Endavant fins detectar
Endavant fins detectar
Endavant fins  no detectar
Endavant fins  no detectar
En